In [42]:
# 1. Librerias
from pprint import pprint
import json
from openai import OpenAI
import time
from dotenv import load_dotenv
import os
import numpy as np
import base64
from Funciones import gather_user_data, gather_reservation_data, not_talk
import requests
from pathlib import Path
import warnings 
warnings.filterwarnings("ignore")
###################################################
# 2. Cargar variables entorno
load_dotenv();
openai_key= os.getenv("OPENAI_KEY");
#print(openai_key)
###################################################
# 3. Crear el cliente (permite la conexión con OPENAI)
client=OpenAI(api_key= openai_key);

In [17]:
# 4. ChatCompletions
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "Eres un asistente util"},
    {"role": "user", "content": "Quien gano la serie mundial de 2020?"},
    {"role": "assistant", "content": "Los Angeles Dodgers"},
    {"role": "user", "content": "Cuando se jugo?"}
  ]
)
response.choices[0].message.content

'La Serie Mundial de 2020 se jugó del 20 al 27 de octubre de ese año.'

In [19]:
# 4.1 JSON mode
response = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": "Eres un asistente util diseñado para responder en formato JSON."},
    {"role": "user", "content": "Quien gano la serie mundial de 2020?"},
    {"role": "assistant", "content": "Los Angeles Dodgers"},
    {"role": "user", "content": "Cuando se jugo?"}
  ]
)
print(response.choices[0].message.content)

		{
			"fecha": "20 de octubre - 27 de octubre de 2020"
		}


In [22]:
# 5. Function calling
messages = [{"role": "user", "content": "What's the weather like in San Francisco, Tokyo, and Paris?"}]
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]
response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    tool_choice="auto",  # auto is default, but we'll be explicit
)
response_message = response.choices[0].message
tool_calls = response_message.tool_calls
print(response_message)
print(tool_calls)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_eHV6bQT6E0Su90abhVf18j4r', function=Function(arguments='{"location": "San Francisco, CA", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_wDyS26sXRfEsLDx4tXb4mEcR', function=Function(arguments='{"location": "Tokyo, Japan", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_P8aPhY0SNTwBRwRNLyFkD0Wr', function=Function(arguments='{"location": "Paris, France", "unit": "celsius"}', name='get_current_weather'), type='function')])
[ChatCompletionMessageToolCall(id='call_eHV6bQT6E0Su90abhVf18j4r', function=Function(arguments='{"location": "San Francisco, CA", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_wDyS26sXRfEsLDx4tXb4mEcR', function=Function(arguments='{"location": "Tokyo, Japan", "unit": "

In [25]:
[x.function for x in response_message.tool_calls]

[Function(arguments='{"location": "San Francisco, CA", "unit": "celsius"}', name='get_current_weather'),
 Function(arguments='{"location": "Tokyo, Japan", "unit": "celsius"}', name='get_current_weather'),
 Function(arguments='{"location": "Paris, France", "unit": "celsius"}', name='get_current_weather')]

In [26]:
tool_calls

[ChatCompletionMessageToolCall(id='call_eHV6bQT6E0Su90abhVf18j4r', function=Function(arguments='{"location": "San Francisco, CA", "unit": "celsius"}', name='get_current_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_wDyS26sXRfEsLDx4tXb4mEcR', function=Function(arguments='{"location": "Tokyo, Japan", "unit": "celsius"}', name='get_current_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_P8aPhY0SNTwBRwRNLyFkD0Wr', function=Function(arguments='{"location": "Paris, France", "unit": "celsius"}', name='get_current_weather'), type='function')]

In [27]:
# 6. Embeddings
response = client.embeddings.create(
    input="Hola como estás",
    model="text-embedding-3-small"
)

print(response.data[0].embedding)

[0.052332207560539246, -0.006519518792629242, -0.03851192817091942, -0.009242862462997437, -0.017044277861714363, -0.00794996228069067, -0.0015143440105021, 0.039062097668647766, -0.05563322827219963, -0.052772343158721924, 0.003928216639906168, 0.029621172696352005, -0.011014411225914955, -0.04469584301114082, 0.00045010814210399985, 0.0669887438416481, -0.028058689087629318, 0.003870448563247919, -0.008753210306167603, 0.056557513773441315, 0.0018458216218277812, -0.013468170538544655, -0.01863977313041687, -0.009925073944032192, 0.03338433802127838, -0.0001770860981196165, -0.015085672028362751, 0.03320828452706337, 0.021577682346105576, -0.006150905042886734, 0.04898717254400253, -0.046566419303417206, 0.00398598425090313, 0.02044433169066906, -0.017429398372769356, 0.009347395040094852, -0.012191775254905224, 0.0356290340423584, 0.02148965559899807, -0.04713859781622887, 0.030919576063752174, -0.006117894779890776, 0.03265811502933502, 0.06417187303304672, 0.02253497950732708, -0.

In [28]:
len(response.data[0].embedding)

1536

In [30]:
def normalize_l2(x):
    x = np.array(x)
    if x.ndim == 1:
        norm = np.linalg.norm(x)
        if norm == 0:
            return x
        return x / norm
    else:
        norm = np.linalg.norm(x, 2, axis=1, keepdims=True)
        return np.where(norm == 0, x, x / norm)


response = client.embeddings.create(
    model="text-embedding-3-small", input="Hola como estás", encoding_format="float"
)

cut_dim = response.data[0].embedding[:256]
norm_dim = normalize_l2(cut_dim)

print(norm_dim)

[ 0.10928511 -0.01366064 -0.08047024 -0.01934778 -0.0356854  -0.01663634
 -0.0031624   0.08148128 -0.11617862 -0.11029616  0.00820902  0.0618118
 -0.02296688 -0.09324621  0.00094786  0.13970847 -0.05850292  0.00814009
 -0.01823333  0.11820072  0.00376845 -0.02796467 -0.03878748 -0.02064606
  0.06971636 -0.00036945 -0.03157228  0.06925679  0.04510653 -0.0128564
  0.10248351 -0.09724445  0.00831242  0.04267083 -0.03644369  0.0195431
 -0.02532216  0.07440396  0.04487675 -0.09853123  0.06470708 -0.01277597
  0.06806192  0.13400983  0.04696778 -0.00996687 -0.04701374  0.03187099
  0.07936728  0.07238185 -0.03506499 -0.03012464  0.07895366  0.3106675
  0.1181088  -0.08336551  0.11911985  0.00280192 -0.00556076  0.00978878
 -0.0317561  -0.04464696  0.05441277 -0.00694521  0.01583209 -0.06930275
 -0.10184012  0.04336017 -0.06728066 -0.00682457 -0.07899962  0.09742827
 -0.04239508 -0.00282921 -0.09926654  0.03894833 -0.05239067 -0.01861248
 -0.01634911  0.04515249 -0.12867885  0.001299   -0.002

In [31]:
len(norm_dim)

256

In [ ]:
# 7. Generacion de imagenes
response = client.images.generate(
  model="dall-e-3",
  prompt="a white siamese cat",
  size="1024x1024",
  quality="standard",
  n=1,
)

image_url = response.data[0].url
image_url

In [34]:
# 8. Vision
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "Berni-Manifestacion.jpg"

# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {openai_key}"
}

payload = {
  "model": "gpt-4o",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "What’s in this image?"
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
  ],
  "max_tokens": 300
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

{'id': 'chatcmpl-9XEc2gTqIVy4tAioWlLcKYOnzv0qi', 'object': 'chat.completion', 'created': 1717708022, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'This image is a painting depicting a group of people gathered together, seemingly in a protest or demonstration. The crowd is diverse, composed of men, women, and a child. The expression on their faces reflects a mix of concern, determination, and resilience. One person in the background holds a sign that reads "PAN Y TRABAJO," which translates to "BREAD AND WORK" in English, suggesting that the crowd is likely demanding basic necessities and employment. The buildings in the background indicate an urban setting. The artwork appears to convey a strong social and political message.'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 778, 'completion_tokens': 114, 'total_tokens': 892}, 'system_fingerprint': 'fp_aa87380ac5'}


In [41]:
response.json()['choices'][0]['message']['content']

'This image is a painting depicting a group of people gathered together, seemingly in a protest or demonstration. The crowd is diverse, composed of men, women, and a child. The expression on their faces reflects a mix of concern, determination, and resilience. One person in the background holds a sign that reads "PAN Y TRABAJO," which translates to "BREAD AND WORK" in English, suggesting that the crowd is likely demanding basic necessities and employment. The buildings in the background indicate an urban setting. The artwork appears to convey a strong social and political message.'

In [47]:
# 9. TTS
speech_file_path = "./speech.mp3"
response = client.audio.speech.create(
  model="tts-1",
  voice="alloy",
  input="Hoy es un dia grandioso para todos"
)

response.stream_to_file(speech_file_path)

C:\Users\david\AppData\Local\Temp\ipykernel_27200\3545250465.py:9: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


In [48]:
# 10. TTS
audio_file= open("./speech.mp3", "rb")
transcription = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file
)
print(transcription.text)

Hoy es un día grandioso para todos.
